In [1]:
import os
import math
import random
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 11})
import matplotlib.ticker as ticker
# PyTorch
import torch
# GPyTorch
from gpytorch.mlls import ExactMarginalLogLikelihood
# BOTorch
import botorch
from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import LogExpectedImprovement
from botorch.optim import optimize_acqf

In [2]:
################################################################################
def check_epochs(df, n, batch_size=128, steps=6000, drop_last=True):
    num_batches = math.floor(n/batch_size) if drop_last else math.ceil(n/batch_size)
    epochs = int(steps/num_batches)
    return df.shape[0] == epochs

def print_job(alpha, beta, dataset, dataset_dir, experiments_dir, lr_0, 
              method, model, model_arch, n, prior_dir, prior_type, random_state, 
              save, tune):
    model_name = f"{model}_alpha={alpha}_beta={beta}_lr_0={lr_0}_n={n}_random_state={random_state}"
    if os.path.exists(f"{experiments_dir}/{model_name}.csv"):
        temp_df = pd.read_csv(f"{experiments_dir}/{model_name}.csv")
        n_train = n - int((1/5) * n) if tune else n
        if check_epochs(temp_df, n_train, batch_size=min(128, n_train), steps=6000, drop_last=True):
            return

    command = (
        f"python ../src/main_image_classifiers.py "
        f"--alpha={alpha} "
        "--batch_size=128 "
        f"--beta={beta} "
        f"--dataset=\"{dataset}\" "
        f"--dataset_dir=\"{dataset_dir}\" "
        f"--experiments_dir=\"{experiments_dir}\" "
        f"--lr_0={lr_0} "
        f"--method=\"{method}\" "
        f"--model=\"{model}\" "
        f"--model_arch=\"{model_arch}\" "
        f"--model_name=\"{model_name}\" "
        f"--n={n} "
        "--num_workers=0 "
        f"--prior_dir=\"{prior_dir}\" "
        f"--prior_type=\"{prior_type}\" "
        f"--random_state={random_state} "
        f"{'--save' if save else ''}"
        f"{'--tune' if tune else ''}"
    )
    
    print(f"    '{command}'")
    
def get_runtime(alpha, beta, experiments_dir, lr_0, model, n, random_state, tune):
    model_name = f"{model}_alpha={alpha}_beta={beta}_lr_0={lr_0}_n={n}_random_state={random_state}"
    if not os.path.exists(f"{experiments_dir}/{model_name}.csv"):
        return 0.0
        #raise FileNotFoundError(f"Expected file not found: {experiments_dir}/{model_name}.csv")
    df = pd.read_csv(f"{experiments_dir}/{model_name}.csv")
    n_train = n - int((1/5) * n) if tune else n
    if not check_epochs(df, n_train, batch_size=min(128, n_train), steps=6000, drop_last=True):
        return 0.0
        #raise RuntimeError(f"Run incomplete: {model_name} did not run for the specified number of epochs")
    return df["train_sec/epoch"].sum()

def get_val_or_test_acc(alpha, beta, experiments_dir, lr_0, model, n, random_state, tune):
    model_name = f"{model}_alpha={alpha}_beta={beta}_lr_0={lr_0}_n={n}_random_state={random_state}"
    if not os.path.exists(f"{experiments_dir}/{model_name}.csv"):
        return 0.0
        #raise FileNotFoundError(f"Expected file not found: {experiments_dir}/{model_name}.csv")
    df = pd.read_csv(f"{experiments_dir}/{model_name}.csv")
    n_train = n - int((1/5) * n) if tune else n
    if not check_epochs(df, n_train, batch_size=min(128, n_train), steps=6000, drop_last=True):
        return 0.0
        #raise RuntimeError(f"Run incomplete: {model_name} did not run for the specified number of epochs")
    return df["val_or_test_acc"].values[-1]
    
def get_val_or_test_nll(alpha, beta, experiments_dir, lr_0, model, n, random_state, tune):
    model_name = f"{model}_alpha={alpha}_beta={beta}_lr_0={lr_0}_n={n}_random_state={random_state}"
    if not os.path.exists(f"{experiments_dir}/{model_name}.csv"):
        return float("inf")
        #raise FileNotFoundError(f"Expected file not found: {experiments_dir}/{model_name}.csv")
    df = pd.read_csv(f"{experiments_dir}/{model_name}.csv")
    n_train = n - int((1/5) * n) if tune else n
    if not check_epochs(df, n_train, batch_size=min(128, n_train), steps=6000, drop_last=True):
        return float("inf")
        #raise RuntimeError(f"Run incomplete: {model_name} did not run for the specified number of epochs")
    return df["val_or_test_nll"].values[-1]

def get_candidate(train_X, train_Y, seed):
    
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    gp = SingleTaskGP(
        train_X=train_X,
        train_Y=train_Y,
        input_transform=Normalize(d=3),
        outcome_transform=Standardize(m=1),
    )
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    logEI = LogExpectedImprovement(model=gp, best_f=train_Y.max())

    bounds = torch.stack([torch.zeros(3), torch.ones(3)]).to(torch.double)
    candidate, acq_value = optimize_acqf(
      logEI, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
    )
    
    return candidate.detach()

In [3]:
# DONE:

# CIFAR-10 n_iters = 41 tuned
# Flower-102 n_iters = 41 tuned
# Pet-37 n_iters = 41 tuned

# CIFAR-10 n_iters = 35 retrained
# Flower-102 n_iters = 35 retrained
# Pet-37 n_iters = 35 retrained

# TODO:

# CIFAR-10 n_iters = 42 tuned
# Flower-102 n_iters = 42 tuned
# Pet-37 n_iters = 42 tuned

# CIFAR-10 n_iters = 36 retrained
# Flower-102 n_iters = 36 retrained
# Pet-37 n_iters = 36 retrained

In [4]:
dataset = "Flower-102"
dataset_dir = "{home_dir}/Flower-102"
model = "l2-sp"
method = "MAP"
model_arch = "ConvNeXt-Tiny"
ns = [510, 1020]
prior_dir = "{home_dir}/convnext_tiny_torchvision"
prior_type = "convnext_tiny_torchvision"
random_states = [1001, 2001, 3001]
retrained_experiments_dir = "{home_dir}/data-emphasized-ELBo/experiments/retrained_Flower-102_ConvNeXt-Tiny_BO"
tuned_experiments_dir = "{home_dir}/data-emphasized-ELBo/experiments/tuned_Flower-102_ConvNeXt-Tiny_BO"

n_iters = 35
seeds = [0, 1, 2, 3, 4]

bounds = torch.tensor([[-6, -6, -4], [-2, -2, -1]], dtype=torch.double)

columns = ["alpha", "beta", "lr_0", "n", "n_iter", "random_state", "runtime", "seed", "val_acc", "val_nll", "test_acc", "test_nll"]
flower102_bo_df = pd.DataFrame(columns=columns)

for n, random_state, seed in itertools.product(ns, random_states, seeds):

    gen = torch.Generator()
    gen.manual_seed(seed)

    train_X = torch.rand(size=(1, 3,), generator=gen, dtype=torch.double)
    train_X_bounded = (bounds[1] - bounds[0]) * train_X + bounds[0]
    
    alpha, beta, lr_0 = 10**train_X_bounded[0]
    print_job(alpha, beta, dataset, dataset_dir, tuned_experiments_dir, lr_0, method, model, model_arch, n, prior_dir, prior_type, random_state, False, True)

    train_Y = torch.tensor([[-get_val_or_test_nll(10**x[0], 10**x[1], tuned_experiments_dir, 10**x[2], model, n, random_state, True)] for x in train_X_bounded], dtype=torch.float64)
    
    alpha_star, beta_star, lr_0_star = 10**train_X_bounded[torch.argmax(train_Y)]
    print_job(alpha_star, beta_star, dataset, dataset_dir, retrained_experiments_dir, lr_0_star, method, model, model_arch, n, prior_dir, prior_type, random_state, True, False)
    
    runtime = get_runtime(alpha, beta, tuned_experiments_dir, lr_0, model, n, random_state, True)
    runtime += get_runtime(alpha_star, beta_star, retrained_experiments_dir, lr_0_star, model, n, random_state, False)
    
    val_acc = get_val_or_test_acc(alpha, beta, tuned_experiments_dir, lr_0, model, n, random_state, True)
    val_nll = get_val_or_test_nll(alpha, beta, tuned_experiments_dir, lr_0, model, n, random_state, True)
    test_acc = get_val_or_test_acc(alpha_star, beta_star, retrained_experiments_dir, lr_0_star, model, n, random_state, False)
    test_nll = get_val_or_test_nll(alpha_star, beta_star, retrained_experiments_dir, lr_0_star, model, n, random_state, False)

    row = [alpha.item(), beta.item(), lr_0.item(), n, 0, random_state, runtime, seed, val_acc, val_nll, test_acc, test_nll]
    flower102_bo_df.loc[len(flower102_bo_df)] = row
    
    for i in range(1, n_iters+1):

        candidate = get_candidate(train_X, train_Y, seed)
        candidate_bounded = (bounds[1] - bounds[0]) * candidate + bounds[0]
        train_X = torch.cat([train_X, candidate])
        train_X_bounded = (bounds[1] - bounds[0]) * train_X + bounds[0]
                
        alpha, beta, lr_0 = 10**candidate_bounded[0]        
        print_job(alpha, beta, dataset, dataset_dir, tuned_experiments_dir, lr_0, method, model, model_arch, n, prior_dir, prior_type, random_state, False, True)

        train_Y = torch.tensor([[-get_val_or_test_nll(10**x[0], 10**x[1], tuned_experiments_dir, 10**x[2], model, n, random_state, True)] for x in train_X_bounded], dtype=torch.float64)

        alpha_star, beta_star, lr_0_star = 10**train_X_bounded[torch.argmax(train_Y)]
        #print_job(alpha_star, beta_star, dataset, dataset_dir, retrained_experiments_dir, lr_0_star, method, model, model_arch, n, prior_dir, prior_type, random_state, True, False)
        
        runtime = sum([get_runtime(10**x[0], 10**x[1], tuned_experiments_dir, 10**x[2], model, n, random_state, True) for x in train_X_bounded])
        runtime += get_runtime(alpha_star, beta_star, retrained_experiments_dir, lr_0_star, model, n, random_state, False)
        
        val_acc = get_val_or_test_acc(alpha, beta, tuned_experiments_dir, lr_0, model, n, random_state, True)
        val_nll = get_val_or_test_nll(alpha, beta, tuned_experiments_dir, lr_0, model, n, random_state, True)
        test_acc = get_val_or_test_acc(alpha_star, beta_star, retrained_experiments_dir, lr_0_star, model, n, random_state, False)
        test_nll = get_val_or_test_nll(alpha_star, beta_star, retrained_experiments_dir, lr_0_star, model, n, random_state, False)

        row = [alpha.item(), beta.item(), lr_0.item(), n, i, random_state, runtime, seed, val_acc, val_nll, test_acc, test_nll]
        flower102_bo_df.loc[len(flower102_bo_df)] = row

flower102_bo_df.head(100)

,alpha,beta,lr_0,n,n_iter,random_state,runtime,seed,val_acc,val_nll,test_acc,test_nll
0,0.007589,0.000678,0.002389,510.0,0.0,1001.0,7108.617203,0.0,0.794118,0.944847,0.832493,0.820326
1,0.000001,0.000001,0.100000,510.0,1.0,1001.0,10784.787583,0.0,0.666667,1.488824,0.832493,0.820326
2,0.003448,0.001479,0.000639,510.0,2.0,1001.0,14534.996344,0.0,0.764706,1.545155,0.832493,0.820326
3,0.010000,0.000513,0.003804,510.0,3.0,1001.0,18374.722632,0.0,0.813726,0.894170,0.849457,0.762403
4,0.010000,0.003177,0.004515,510.0,4.0,1001.0,22179.254153,0.0,0.813726,0.927907,0.849457,0.762403
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000018,0.000001,0.020334,510.0,23.0,1001.0,93540.990056,2.0,0.843137,0.668450,0.892671,0.518188
96,0.000001,0.000001,0.017856,510.0,24.0,1001.0,97861.830223,2.0,0.803922,0.711648,0.892671,0.518188
97,0.000040,0.000001,0.022994,510.0,25.0,1001.0,102166.745179,2.0,0.843137,0.625879,0.893234,0.514453
98,0.010000,0.000001,0.001238,510.0,26.0,1001.0,105892.779017,2.0,0.794118,1.124442,0.893234,0.514453


In [5]:
#flower102_bo_df.to_csv("Flower-102_ConvNeXt-Tiny_BO.csv", index=False)
flower102_bo_df = pd.read_csv("Flower-102_ConvNeXt-Tiny_BO.csv")
flower102_bo_df.head(100)

,alpha,beta,lr_0,n,n_iter,random_state,runtime,seed,val_acc,val_nll,test_acc,test_nll
0,0.007589,0.000678,0.002389,510.0,0.0,1001.0,7108.617203,0.0,0.794118,0.944847,0.832493,0.820326
1,0.000001,0.000001,0.100000,510.0,1.0,1001.0,10784.787583,0.0,0.666667,1.488824,0.832493,0.820326
2,0.003448,0.001479,0.000639,510.0,2.0,1001.0,14534.996344,0.0,0.764706,1.545155,0.832493,0.820326
3,0.010000,0.000513,0.003804,510.0,3.0,1001.0,18374.722632,0.0,0.813726,0.894170,0.849457,0.762403
4,0.010000,0.003177,0.004515,510.0,4.0,1001.0,22179.254153,0.0,0.813726,0.927907,0.849457,0.762403
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000018,0.000001,0.020334,510.0,23.0,1001.0,93540.990056,2.0,0.843137,0.668450,0.892671,0.518188
96,0.000001,0.000001,0.017856,510.0,24.0,1001.0,97861.830223,2.0,0.803922,0.711648,0.892671,0.518188
97,0.000040,0.000001,0.022994,510.0,25.0,1001.0,102166.745179,2.0,0.843137,0.625879,0.893234,0.514453
98,0.010000,0.000001,0.001238,510.0,26.0,1001.0,105892.779017,2.0,0.794118,1.124442,0.893234,0.514453
